In [1]:
import sys, os

import torch
import torch.nn as nn
from torch.utils.data import (
    Dataset, RandomSampler, DataLoader)

from dataset.mnist import load_mnist


sys.path.append(os.pardir)

(x_train, y_train), (x_test, y_test) = load_mnist(flatten=True, normalize=True, one_hot_label=False)

x_train, y_train, x_test, y_test = (
    torch.Tensor(d) for d in (x_train, y_train, x_test, y_test)
)

y_train = y_train.long()
y_test = y_test.long()

In [2]:
class MnistDataset(Dataset):
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __len__(self):
        return self.x.size(0)
    
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [3]:
train_dataset = MnistDataset(x_train, y_train)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = MnistDataset(x_test, y_test)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [4]:
# device = 'cpu'
device = 'cuda'

In [5]:
model = nn.Sequential(
    nn.Linear(784, 100, bias=True), nn.ReLU(),
    nn.Linear(100, 50, bias=True), nn.ReLU(),
    nn.Linear(50, 10, bias=True), nn.Softmax(dim=-1)
).to(device)

In [6]:
optimizer = torch.optim.SGD(model.parameters(), lr=1e-1, momentum=0.5)

In [8]:
loss_fn = nn.CrossEntropyLoss()

In [9]:
EPOCHS = 100
n = 60000
bsz = train_dataloader.batch_size
for i in range(EPOCHS):
    correct = 0
    loss = 0
    for batch_idx, sample in enumerate(train_dataloader):
        x, y = sample[0].to(device), sample[1].to(device)
        optimizer.zero_grad()
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        loss.backward()
        optimizer.step()
        correct += torch.eq(torch.argmax(y_pred, dim=-1), y).sum().item()
        loss += loss.item()
    print(f'\rEPOCH: [{i+1:03} / {EPOCHS:03}] Acc={correct/n:.2%} Loss={loss/bsz:.4f}')

EPOCH: [001 / 100] Acc=61.12% Loss=0.0517
EPOCH: [002 / 100] Acc=86.38% Loss=0.0489
EPOCH: [003 / 100] Acc=91.42% Loss=0.0499
EPOCH: [004 / 100] Acc=92.74% Loss=0.0494
EPOCH: [005 / 100] Acc=93.75% Loss=0.0481
EPOCH: [006 / 100] Acc=94.59% Loss=0.0477
EPOCH: [007 / 100] Acc=95.26% Loss=0.0461
EPOCH: [008 / 100] Acc=95.91% Loss=0.0461
EPOCH: [009 / 100] Acc=96.23% Loss=0.0468
EPOCH: [010 / 100] Acc=96.61% Loss=0.0457
EPOCH: [011 / 100] Acc=96.92% Loss=0.0471
EPOCH: [012 / 100] Acc=97.25% Loss=0.0458
EPOCH: [013 / 100] Acc=97.42% Loss=0.0467
EPOCH: [014 / 100] Acc=97.60% Loss=0.0457
EPOCH: [015 / 100] Acc=97.83% Loss=0.0474
EPOCH: [016 / 100] Acc=97.97% Loss=0.0457
EPOCH: [017 / 100] Acc=98.07% Loss=0.0458
EPOCH: [018 / 100] Acc=98.16% Loss=0.0457
EPOCH: [019 / 100] Acc=98.27% Loss=0.0466
EPOCH: [020 / 100] Acc=98.35% Loss=0.0457
EPOCH: [021 / 100] Acc=98.45% Loss=0.0459
EPOCH: [022 / 100] Acc=98.54% Loss=0.0467
EPOCH: [023 / 100] Acc=98.55% Loss=0.0457
EPOCH: [024 / 100] Acc=98.62% Loss

In [10]:
model.eval()

Sequential(
  (0): Linear(in_features=784, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=50, bias=True)
  (3): ReLU()
  (4): Linear(in_features=50, out_features=10, bias=True)
  (5): Softmax(dim=-1)
)

In [11]:
correct = 0
for sample in test_dataloader:
    x, y = sample[0].to(device), sample[1].to(device)
    y_pred = model(x)
    correct += torch.eq(torch.argmax(y_pred, dim=-1), y).sum().item()

In [12]:
correct / y_test.size(0)

0.9768